In [83]:
from xgboost import XGBClassifier
# read data
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(data['data'], data['target'], test_size=.2, random_state=42)
# create model instance
xgbc_model = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')
# fit model
xgbc_model.fit(X_train, y_train)
# make predictions
y_pred = xgbc_model.predict(X_test)

xgbc_model.save_model('xgbc_iris.model')

print(X_test[:5])
print(y_test[:5])
print(y_pred[:5])

[[6.1 2.8 4.7 1.2]
 [5.7 3.8 1.7 0.3]
 [7.7 2.6 6.9 2.3]
 [6.  2.9 4.5 1.5]
 [6.8 2.8 4.8 1.4]]
[1 0 2 1 1]
[1 0 2 1 1]


In [84]:
# !pip3 install onnx
# !pip3 install onnxconverter_common
# !pip3 install onnxmltools
# !pip3 install onnxruntime
# !pip3 install pyquickhelper
!pip3 install docutils
!pip3 install mlprodict

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [87]:
from pyquickhelper.helpgen.graphviz_helper import plot_graphviz
from mlprodict.onnxrt import OnnxInference
import numpy
import onnxruntime as rt
from sklearn.datasets import load_iris, load_diabetes, make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier, XGBRegressor, DMatrix, train as train_xgb
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert_sklearn, to_onnx, update_registered_converter
from skl2onnx.common.shape_calculator import (
    calculate_linear_classifier_output_shapes,
    calculate_linear_regressor_output_shapes)
from onnxmltools.convert.xgboost.operator_converters.XGBoost import (
    convert_xgboost)
from onnxmltools.convert import convert_xgboost as convert_xgboost_booster

xgbc = XGBClassifier()
xgbc.load_model("xgbc_iris.model")

pipe = Pipeline([('xgb', xgb_from_model)])

pipe.fit(X_test, y_test)


Pipeline(steps=[('xgb',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=None, gpu_id=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=2, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=2,
                               n_jobs=None, num_parallel_tree=None,
                               objective='multi:softprob', predictor=None, ...))])

In [89]:
update_registered_converter(
    XGBClassifier, 'XGBoostXGBClassifier',
    calculate_linear_classifier_output_shapes, convert_xgboost,
    options={'nocl': [True, False], 'zipmap': [True, False, 'columns']})

model_onnx = convert_sklearn(
    pipe, 'pipeline_xgboost',
    [('input', FloatTensorType([None, 4]))],
    target_opset={'': 12, 'ai.onnx.ml': 4})

# And save.
with open("xgbc_iris.onnx", "wb") as f:
    f.write(model_onnx.SerializeToString())

sess = rt.InferenceSession("xgbc_iris.onnx")
pred_onx = sess.run(None, {"input": X_test[:5].astype(numpy.float32)})
print("predict", pred_onx[0])
print("predict_proba", pred_onx[1][:1])

predict [1 0 2 1 1]
predict_proba [{0: 0.05465603619813919, 1: 0.8903928995132446, 2: 0.0549510195851326}]


In [96]:
!pip install onnx-tf

Defaulting to user installation because normal site-packages is not writeable


In [97]:
!python -m onnxruntime.tools.convert_onnx_models_to_ort xgbc_iris.onnx

Converting models with optimization style 'Fixed' and level 'all'
Converting optimized ONNX model /home/sol/notebook/xgboost/xgbc_iris.onnx to ORT format model /home/sol/notebook/xgboost/xgbc_iris.ort
Converted 1/1 models successfully.
Generating config file from ORT format models with optimization style 'Fixed' and level 'all'
2023-06-10 20:16:27,024 ort_format_model.utils [INFO] - Created config in /home/sol/notebook/xgboost/xgbc_iris.required_operators.config
Converting models with optimization style 'Runtime' and level 'all'
Converting optimized ONNX model /home/sol/notebook/xgboost/xgbc_iris.onnx to ORT format model /home/sol/notebook/xgboost/xgbc_iris.with_runtime_opt.ort
Converted 1/1 models successfully.
Converting models again without runtime optimizations to generate a complete config file. These converted models are temporary and will be deleted.
Converting optimized ONNX model /home/sol/notebook/xgboost/xgbc_iris.onnx to ORT format model /home/sol/notebook/xgboost/tmp9egpu7

# Note: cannot convert xgboost model (.onnx) to tensorflow model

## Conversion code

```
import onnx
from onnx_tf.backend import prepare

# Load the ONNX model
onnx_model = onnx.load('xgbc_iris.onnx')

# Convert the ONNX model to TensorFlow format
tf_model = prepare(onnx_model)

# Save the TensorFlow model
tf_model.export_graph('xgbc_iris.tf')
```

## Conversion error

```
# Error: "BackendIsNotSupposedToImplementIt: TreeEnsembleClassifier is not implemented."
```